# Zadanie: Implementacja Drzewa Decyzyjnego dla Systemu Rekomendacji Kredytowej

## Cel zadania

Celem zadania jest implementacja drzewa decyzyjnego do oceny wniosków kredytowych. System powinien na podstawie różnych parametrów wnioskodawcy podejmować decyzję o przyznaniu kredytu oraz określać jego warunki.

## Opis zadania

Stwórz drzewo decyzyjne, które będzie analizować następujące parametry:
1. Dochód miesięczny
2. Historia kredytowa (scoring)
3. Staż pracy
4. Aktualne zobowiązania
5. Wartość wnioskowanego kredytu

Na podstawie tych parametrów system powinien:
- Podjąć decyzję o przyznaniu/odmowie kredytu
- Określić maksymalną kwotę kredytu
- Zaproponować okres spłaty
- Ustalić oprocentowanie

## Struktura implementacji

In [ ]:
from dataclasses import dataclass
from typing import Optional, Dict, Any

@dataclass
class CreditApplication:
    monthly_income: float
    credit_score: int  # 0-100
    employment_years: float
    current_debts: float
    requested_amount: float

@dataclass
class CreditDecision:
    approved: bool
    max_amount: float
    period_months: int
    interest_rate: float
    reason: str

class DecisionNode:
    def __init__(self, feature: str, threshold: float, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left  # węzeł dla wartości <= threshold
        self.right = right  # węzeł dla wartości > threshold
        self.value = value  # wynik dla węzła końcowego (liścia)
    
    def is_leaf(self) -> bool:
        return self.value is not None

class CreditDecisionTree:
    def __init__(self):
        self.root = self._build_tree()
        self.min_credit_score = 50
        self.max_debt_to_income = 0.4
        
    def _build_tree(self) -> DecisionNode:
        """Buduje drzewo decyzyjne na podstawie reguł biznesowych"""
        # Przykładowa struktura drzewa - do rozbudowy
        return DecisionNode(
            'credit_score',
            self.min_credit_score,
            left=DecisionNode(None, None, value={'approved': False, 'reason': 'Zbyt niski scoring kredytowy'}),
            right=DecisionNode(
                'debt_ratio',
                self.max_debt_to_income,
                left=self._build_amount_subtree(),
                right=DecisionNode(None, None, value={'approved': False, 'reason': 'Zbyt wysokie zadłużenie'})
            )
        )
    
    def _build_amount_subtree(self) -> DecisionNode:
        """Buduje poddrzewo decyzyjne dla kwoty kredytu"""
        # TODO: Zaimplementuj logikę oceny kwoty kredytu
        pass

    def calculate_metrics(self, application: CreditApplication) -> Dict[str, float]:
        """Oblicza metryki potrzebne do podjęcia decyzji"""
        monthly_debt_payment = application.current_debts * 0.05  # Uproszczone obliczenie raty
        debt_ratio = monthly_debt_payment / application.monthly_income
        income_multiplier = application.monthly_income * 12 * application.employment_years
        
        return {
            'debt_ratio': debt_ratio,
            'income_multiplier': income_multiplier,
            'monthly_free_income': application.monthly_income - monthly_debt_payment
        }

    def _traverse_tree(self, node: DecisionNode, features: Dict[str, Any]) -> Dict[str, Any]:
        """Przechodzi przez drzewo decyzyjne"""
        if node.is_leaf():
            return node.value
            
        if features[node.feature] <= node.threshold:
            return self._traverse_tree(node.left, features)
        return self._traverse_tree(node.right, features)

    def evaluate_application(self, application: CreditApplication) -> CreditDecision:
        """Ocenia wniosek kredytowy i podejmuje decyzję"""
        # Obliczanie metryk
        metrics = self.calculate_metrics(application)
        
        # Łączenie danych aplikacji z metrykami
        features = {
            'credit_score': application.credit_score,
            'debt_ratio': metrics['debt_ratio'],
            'monthly_income': application.monthly_income,
            'employment_years': application.employment_years,
            'requested_amount': application.requested_amount
        }
        
        # Przejście przez drzewo
        decision = self._traverse_tree(self.root, features)
        
        if not decision['approved']:
            return CreditDecision(
                approved=False,
                max_amount=0,
                period_months=0,
                interest_rate=0,
                reason=decision['reason']
            )
        
        # Obliczanie warunków kredytu
        max_amount = min(
            application.requested_amount,
            metrics['monthly_free_income'] * 60  # Maksymalnie 5 lat
        )
        
        # Ustalanie oprocentowania na podstawie scoringu
        base_rate = 0.05  # 5%
        risk_premium = (100 - application.credit_score) / 1000  # 0-10%
        interest_rate = base_rate + risk_premium
        
        # Ustalanie okresu kredytowania
        period_months = self._calculate_optimal_period(max_amount, metrics['monthly_free_income'], interest_rate)
        
        return CreditDecision(
            approved=True,
            max_amount=max_amount,
            period_months=period_months,
            interest_rate=interest_rate,
            reason="Wniosek zaakceptowany"
        )
    
    def _calculate_optimal_period(self, amount: float, monthly_free_income: float, interest_rate: float) -> int:
        """Oblicza optymalny okres kredytowania"""
        # TODO: Zaimplementuj obliczanie optymalnego okresu
        # Powinno uwzględniać:
        # - Maksymalną ratę (nie więcej niż 70% wolnego dochodu)
        # - Preferencję krótszych okresów
        # - Minimalizację całkowitego kosztu kredytu
        pass

## Przykład użycia

In [ ]:
# Inicjalizacja systemu
decision_system = CreditDecisionTree()

# Przykładowy wniosek
application = CreditApplication(
    monthly_income=5000,
    credit_score=75,
    employment_years=3,
    current_debts=10000,
    requested_amount=50000
)

# Ocena wniosku
decision = decision_system.evaluate_application(application)

print(f"Decyzja: {'Pozytywna' if decision.approved else 'Negatywna'}")
if decision.approved:
    print(f"Maksymalna kwota: {decision.max_amount:.2f} PLN")
    print(f"Okres kredytowania: {decision.period_months} miesięcy")
    print(f"Oprocentowanie: {decision.interest_rate*100:.2f}%")
print(f"Uzasadnienie: {decision.reason}")

## Zadania do wykonania

1. Rozbuduj drzewo decyzyjne o:
   - Bardziej szczegółową analizę historii kredytowej
   - Uwzględnienie typu zatrudnienia
   - Analizę wieku wnioskodawcy
   - Ocenę zabezpieczeń kredytu

2. Zaimplementuj funkcję _build_amount_subtree():
   - Dodaj różne progi kwotowe
   - Uwzględnij relację kwoty do dochodu
   - Zaimplementuj różne ścieżki decyzyjne

3. Zaimplementuj funkcję _calculate_optimal_period():
   - Obliczanie rat kredytu
   - Optymalizacja okresu kredytowania
   - Uwzględnienie preferencji klienta

4. Dodaj system walidacji i weryfikacji:
   - Sprawdzanie poprawności danych wejściowych
   - Weryfikacja wewnętrznej spójności decyzji
   - Logi decyzji i uzasadnienia

5. Rozszerz system o:
   - Różne produkty kredytowe
   - System scoringowy
   - Analizę ryzyka
   - Rekomendacje alternatywnych produktów